In [1]:
#hide
from nbdev.showdoc import show_doc

# 10-fold Cross Validation
> Code for training is the same as Notebook 05.

In [2]:
from car_speech.fname_processing import load_fnames
from car_speech.pipeline import *

import string
from random import shuffle
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import layers
from tensorflow.keras import models
from sklearn.model_selection import train_test_split, KFold

## Configuration

In [3]:
DATASET_TYPE = 'digits' # 'digits' or 'letters' or 'mixed'
LEVEL_NAME = 'IDL' # 'IDL' or '35U' or '35D' or '55U' or '55D'

EPOCHS = 100

In [4]:
if DATASET_TYPE == 'digits':
    label_strings = np.array([str(num) for num in range(0,10)])
    fname_str = 'digit'
elif DATASET_TYPE == 'letters':
    label_strings = np.array(list(string.ascii_uppercase))
    fname_str = 'letter'
elif DATASET_TYPE == 'mixed':
    label_strings = np.array([str(num) for num in range(0,10)] + list(string.ascii_uppercase))
    fname_str = 'mixed'
    
# load classified filenames
filenames = load_fnames('noise_levels/' + fname_str + '_noise_levels/' + LEVEL_NAME + '.data')
num_samples = len(filenames)
print('number of files:', len(filenames))

# shuffle
shuffle(filenames)

fold_no = 1
acc_per_fold = []

number of files: 2677


## Loop for 10-fold

In [5]:
beg_ratio = 0
while beg_ratio < 0.9:
    test_beg = int(beg_ratio*num_samples)
    test_end = int((beg_ratio+0.1)*num_samples)
    
    fnames_copy = deepcopy(filenames)
    test_files = fnames_copy[test_beg:test_end]
    del fnames_copy[test_beg:test_end]
    
    val_len = len(test_files)
    val_files = fnames_copy[-val_len:]
    del fnames_copy[-val_len:]
    
    train_files = fnames_copy
    
    beg_ratio += 0.1
    
    train_files = shuffle_data(train_files)
    val_files = shuffle_data(val_files)
    test_files = shuffle_data(test_files)
    

    # Process data using the combined pipeline
    spectrogram_ds = preprocess_dataset(train_files, DATASET_TYPE)
    train_ds = spectrogram_ds
    val_ds = preprocess_dataset(val_files, DATASET_TYPE)
    test_ds = preprocess_dataset(test_files, DATASET_TYPE)
    print("Pipeline Completed")
    
    # batch
    batch_size = 64
    train_ds = train_ds.batch(batch_size)
    val_ds = val_ds.batch(batch_size)
    
    AUTOTUNE = tf.data.experimental.AUTOTUNE
    train_ds = train_ds.cache().prefetch(AUTOTUNE)
    val_ds = val_ds.cache().prefetch(AUTOTUNE)
    
    # model
    for spectrogram, _ in spectrogram_ds.take(1):
        input_shape = spectrogram.shape
#     print('Input shape:', input_shape)
    num_labels = len(label_strings)

    norm_layer = preprocessing.Normalization()
    norm_layer.adapt(spectrogram_ds.map(lambda x, _: x))

#     model = models.Sequential([
#         layers.Input(shape=input_shape),
#         preprocessing.Resizing(32, 32), 
#         norm_layer,
#         layers.Conv2D(32, 3, activation='relu'),
#         layers.Conv2D(64, 3, activation='relu'),
#         layers.MaxPooling2D(),
#         layers.Dropout(0.25),
#         layers.Flatten(),
#         layers.Dense(128, activation='relu'),
#         layers.Dropout(0.5),
#         layers.Dense(num_labels),
#     ])

    model = models.Sequential([
        layers.Input(shape=input_shape),
        preprocessing.Resizing(32, 32), 
        norm_layer,
        layers.Conv2D(128, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Dropout(0.25),
        layers.Conv2D(128, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Dropout(0.25),
        layers.Conv2D(128, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Dropout(0.25),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_labels),
    ])
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy'],
    )
    
    # Train
    history = model.fit(
        train_ds, 
        validation_data=val_ds,  
        epochs=EPOCHS,
        callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=3),
    )
    
    # Test
    test_audio = []
    test_labels = []

    for audio, label in test_ds:
        test_audio.append(audio.numpy())
        test_labels.append(label.numpy())

    test_audio = np.array(test_audio)
    test_labels = np.array(test_labels)
    
    y_pred = np.argmax(model.predict(test_audio), axis=1)
    y_true = test_labels

    test_acc = sum(y_pred == y_true) / len(y_true)
    print(f'Fold {fold_no} - Test set accuracy: {test_acc:.0%}')
    acc_per_fold.append(round(test_acc, 2))
    fold_no += 1

print(acc_per_fold)
print('mean score:', round(np.mean(acc_per_fold), 2))

Pipeline Completed
Epoch 1/100
34/34 [==============================] - 6s 174ms/step - loss: 3.2555 - accuracy: 0.0532 - val_loss: 3.2160 - val_accuracy: 0.0487
Epoch 2/100
34/34 [==============================] - 4s 132ms/step - loss: 3.1845 - accuracy: 0.0700 - val_loss: 3.0995 - val_accuracy: 0.0787
Epoch 3/100
34/34 [==============================] - 4s 125ms/step - loss: 3.0935 - accuracy: 0.0849 - val_loss: 3.0073 - val_accuracy: 0.0974
Epoch 4/100
34/34 [==============================] - 4s 126ms/step - loss: 3.0126 - accuracy: 0.0980 - val_loss: 2.9042 - val_accuracy: 0.1199
Epoch 5/100
34/34 [==============================] - 4s 118ms/step - loss: 2.9480 - accuracy: 0.1134 - val_loss: 2.7988 - val_accuracy: 0.1386
Epoch 6/100
34/34 [==============================] - 5s 134ms/step - loss: 2.8675 - accuracy: 0.1321 - val_loss: 2.7338 - val_accuracy: 0.1536
Epoch 7/100
34/34 [==============================] - 5s 139ms/step - loss: 2.7600 - accuracy: 0.1475 - val_loss: 2.6301 - v

34/34 [==============================] - 4s 132ms/step - loss: 2.6035 - accuracy: 0.1825 - val_loss: 2.4528 - val_accuracy: 0.2584
Epoch 10/100
34/34 [==============================] - 4s 127ms/step - loss: 2.5125 - accuracy: 0.2133 - val_loss: 2.3889 - val_accuracy: 0.2659
Epoch 11/100
34/34 [==============================] - 4s 126ms/step - loss: 2.4191 - accuracy: 0.2287 - val_loss: 2.3631 - val_accuracy: 0.2247
Epoch 12/100
34/34 [==============================] - 4s 125ms/step - loss: 2.3511 - accuracy: 0.2515 - val_loss: 2.3111 - val_accuracy: 0.2584
Epoch 13/100
34/34 [==============================] - 4s 125ms/step - loss: 2.2848 - accuracy: 0.2613 - val_loss: 2.1911 - val_accuracy: 0.2959
Epoch 14/100
34/34 [==============================] - 4s 125ms/step - loss: 2.2069 - accuracy: 0.2912 - val_loss: 2.1609 - val_accuracy: 0.3146
Epoch 15/100
34/34 [==============================] - 4s 126ms/step - loss: 2.1582 - accuracy: 0.2944 - val_loss: 2.1292 - val_accuracy: 0.3184
Epoch

Epoch 8/100
34/34 [==============================] - 4s 126ms/step - loss: 2.7108 - accuracy: 0.1540 - val_loss: 2.5505 - val_accuracy: 0.2434
Epoch 9/100
34/34 [==============================] - 4s 120ms/step - loss: 2.6186 - accuracy: 0.1839 - val_loss: 2.4615 - val_accuracy: 0.2772
Epoch 10/100
34/34 [==============================] - 4s 127ms/step - loss: 2.5082 - accuracy: 0.2151 - val_loss: 2.3165 - val_accuracy: 0.2996
Epoch 11/100
34/34 [==============================] - 4s 126ms/step - loss: 2.4749 - accuracy: 0.2198 - val_loss: 2.3135 - val_accuracy: 0.2584
Epoch 12/100
34/34 [==============================] - 4s 125ms/step - loss: 2.4096 - accuracy: 0.2389 - val_loss: 2.2483 - val_accuracy: 0.2884
Epoch 13/100
34/34 [==============================] - 4s 125ms/step - loss: 2.2550 - accuracy: 0.2678 - val_loss: 2.1701 - val_accuracy: 0.3034
Epoch 14/100
34/34 [==============================] - 4s 127ms/step - loss: 2.1982 - accuracy: 0.2996 - val_loss: 2.0726 - val_accuracy: 0

Epoch 4/100
34/34 [==============================] - 4s 116ms/step - loss: 3.1061 - accuracy: 0.0892 - val_loss: 2.9919 - val_accuracy: 0.1157
Epoch 5/100
34/34 [==============================] - 4s 124ms/step - loss: 3.0177 - accuracy: 0.0897 - val_loss: 2.8838 - val_accuracy: 0.1306
Epoch 6/100
34/34 [==============================] - 4s 124ms/step - loss: 2.9043 - accuracy: 0.1168 - val_loss: 2.8322 - val_accuracy: 0.1343
Epoch 7/100
34/34 [==============================] - 4s 127ms/step - loss: 2.8170 - accuracy: 0.1369 - val_loss: 2.7128 - val_accuracy: 0.1940
Epoch 8/100
34/34 [==============================] - 4s 118ms/step - loss: 2.6915 - accuracy: 0.1831 - val_loss: 2.6737 - val_accuracy: 0.1828
Epoch 9/100
34/34 [==============================] - 4s 114ms/step - loss: 2.6176 - accuracy: 0.1878 - val_loss: 2.5085 - val_accuracy: 0.2649
Epoch 10/100
34/34 [==============================] - 4s 116ms/step - loss: 2.4814 - accuracy: 0.2335 - val_loss: 2.3691 - val_accuracy: 0.283

In [6]:
print("Done")
print(acc_per_fold)
print('mean score:', round(np.mean(acc_per_fold), 2))

Done
[0.48, 0.43, 0.4, 0.43, 0.39, 0.43, 0.44, 0.45, 0.47, 0.41]
mean score: 0.43
